# CNN Model 2
## Selected images that met the (26, 26) and (25, 27) dimensions. Resized all of them to (26, 26). Removed none defects. No data augmentation. Achieved 0.94 accuracy

Sources:
1. Keras Wafer Classification 2D: TEEEADY https://www.kaggle.com/teeeady/keras-wafer-classification-cnn2d


In [1]:
import os
from os.path import join

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import warnings

!pip install tensorflow
import tensorflow as tf
import keras
from keras import layers, Input, models
from tensorflow.keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow.keras import datasets, layers, models, losses

# Import modeling libraries.
import sklearn as sk
#sklearn.__version__
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from skimage.transform import resize as sk_resize

datapath = join('data', 'wafer')

warnings.filterwarnings("ignore")

In [2]:
# Read the full data file that is stored locally. Look at information and data details

df=pd.read_pickle("LSWMD.pkl")
df.info()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811457 entries, 0 to 811456
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   waferMap        811457 non-null  object 
 1   dieSize         811457 non-null  float64
 2   lotName         811457 non-null  object 
 3   waferIndex      811457 non-null  float64
 4   trianTestLabel  811457 non-null  object 
 5   failureType     811457 non-null  object 
dtypes: float64(2), object(4)
memory usage: 37.1+ MB


,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
571059,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...",710.0,lot35759,19.0,[],[]
796649,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0,...",561.0,lot46939,16.0,[[Test]],[[none]]
770148,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1376.0,lot45860,12.0,[[Test]],[[none]]
701482,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2,...",516.0,lot43084,4.0,[[Training]],[[none]]
766914,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1376.0,lot45729,19.0,[[Test]],[[none]]


In [3]:
#Data Pre-Processing

# image size drop
def find_dim(x):
    dim0=np.size(x,axis=0)
    dim1=np.size(x,axis=1)
    return dim0,dim1

# Data preprocessing process
def __Inputdf__(Height, Width):
    sub_df = df.loc[df['waferMapDim'] == (Height, Width)]
    sw = np.ones((1,  Height, Width))
    label = list()

    for i in range(len(sub_df)):
        if len(sub_df.iloc[i, :]['failureType']) == 0:
            continue
        sw = np.concatenate((sw, sub_df.iloc[i,:]['waferMap'].reshape(1, Height, Width)))
        label.append(sub_df.iloc[i, :]['failureType'][0][0])
    
    x = sw[1:]
    y = np.array(label).reshape((-1,1))
    
    #add channel
    x = x.reshape((-1, Height, Width, 1))
    new_x = np.zeros((len(x), Height, Width, 3))

    for w in range(len(x)):
        for i in range(Height):
            for j in range(Width):
                new_x[w, i, j, int(x[w, i, j])] = 1
    return new_x, y

# Scaling the images
def __InputTf1__(new_x):
    new_result = tf.compat.v1.image.resize(new_x, (26,26),
                                           method=tf.image.ResizeMethod.NEAREST_NEIGHBOR, 
                                           align_corners=True,
                                           preserve_aspect_ratio=False, name=None)
    return new_result  

In [4]:
#We do not need wafer index in our model. Dropping it
df = df.drop(['waferIndex'], axis = 1)

#Since wafer maps have different data lengths, find dimensions
df['waferMapDim']=df.waferMap.apply(find_dim)
df.sample(5)

,waferMap,dieSize,lotName,trianTestLabel,failureType,waferMapDim
386917,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1080.0,lot23173,[],[],"(35, 40)"
458660,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,...",712.0,lot28256,[],[],"(32, 29)"
180605,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2937.0,lot11530,[],[],"(62, 61)"
403482,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...",811.0,lot24160,[],[],"(34, 31)"
211381,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",14116.0,lot13479,[],[],"(212, 84)"


Get sub wafer with resolution.

Wafer is resized to have (26, 26) resolution. rearrange wafer nd-array with faulty case label. some wafer has null label, skip it. First, let's create two types of resolution data by combining them.

In [ ]:
%%time
new_x1, y1 = __Inputdf__(26, 26)
new_x2, y2 = __Inputdf__(25, 27)

In [ ]:
#Resized using Compat.V1 Pre-processing method¶

new_Tf1_1 = __InputTf1__(new_x1)
new_Tf2_1 = __InputTf1__(new_x2)

In [ ]:
# y = y1 + y2
y = np.concatenate((y1, y2), axis=0)
# new_x = new_x1 + new_x2
new_x= tf.concat([new_Tf1_1, new_Tf2_1], 0)

In [ ]:
# Using random sampling as a y array to put the none value into the none_idx variable
none_idx = np.where(y=='none')[0][np.random.choice(len(np.where(y=='none')[0]), size=11000, replace=False)]

# delete the none value
new_x = np.delete(new_x, none_idx, axis=0)
new_y = np.delete(y, none_idx, axis=0)

faulty_case = np.unique(y)
# make string label data to numerical data
for i, l in enumerate(faulty_case):
    new_y[new_y==l] = i
    
# one-hot-encoding
new_y = to_categorical(new_y)

In [ ]:
# split data train, test
x_train, x_test, y_train, y_test = train_test_split(new_x,new_y,test_size=0.33,random_state=2021)

In [ ]:
print('Train x : {}, y : {}'.format(x_train.shape, y_train.shape))
print('Test x: {}, y : {}'.format(x_test.shape, y_test.shape))

In [ ]:
# parameter
epoch=10
batch_size=1024

In [ ]:
def create_model():
    input_shape = (26, 26, 3)
    input_tensor = Input(input_shape)

    conv_1 = layers.Conv2D(16, (3,3), activation='relu', padding='same')(input_tensor)

    flat = layers.Flatten()(conv_1)

    dense_1 = layers.Dense(16, activation='relu')(flat)
    output_tensor = layers.Dense(9, activation='softmax')(dense_1)

    model = models.Model(input_tensor, output_tensor)
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

    return model

In [ ]:
# Make keras model to sklearn classifier.
model = KerasClassifier(build_fn=create_model, epochs=5, batch_size=2048, verbose=2) 
# 3-Fold Crossvalidation
kfold = KFold(n_splits=3, shuffle=True, random_state=2019) 
results = cross_val_score(model, x_train, y_train, cv=kfold)
# Check 3-fold model's mean accuracy
print('Simple CNN Cross validation score : {:.4f}'.format(np.mean(results)))

In [ ]:
history = model.fit(x_train, y_train,validation_data=(x_test, y_test),epochs=epoch,batch_size=batch_size)

In [ ]:
# accuracy plot 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()